In [1]:
import numpy as np
source1 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca.txt"
source2 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-pca-cls.txt"
source3 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd.txt"
source4 = "F:/MSc DS/Project/dms/trained_dms_txt/bert2dm-svd-cls.txt"
source5 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c5.txt"
source6 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-c10.txt"
source7 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d5.txt"
source8 = "F:/MSc DS/Project/dms/trained_dms_txt/ms-word2dm-d10.txt"
source9 = "F:/MSc DS/Project/dms/trained_dms_txt/word2dm.txt"

def get_dict_from_source(source = source5):
    dim =17
    if source ==source1 or source==source3:
        dim = 15
    whole =[]

    with open(source) as f:
        for line in f:
            linelist = line.split()
            for a in linelist:
                whole.append(a)
    chunk = dim**2+1
    n = len(whole)/chunk
    voc = []
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        voc.append(whole[i * chunk])
    dic = {}
    for i in range(int(n)):
        a = whole[i * chunk + 1:(i + 1) * chunk]
        a = np.array(a, dtype='float')
        a.resize([dim, dim])
        dic[voc[i]] = a

    return dic


def ke(a, b):
    dim = a.shape[0]
    m = b - a
    eig, _ = np.linalg.eig(m)
    n = len(eig)
    for i in range(n):
        if eig[i] >= 0:
            eig[i] = 0
        else:
            eig[i] = -eig[i]
    e = np.eye(dim, dtype=float)
    for i in range(dim):
        e[i][i] = eig[i]

    Ke = 1 - np.linalg.norm(e) / np.linalg.norm(a)
    return Ke

def compute_Ke(listA,listB, dic, method ='add'):
    '''
    imput two phrase or sentences lists
    tokenize each pair and get new dm using composition method
    compute Ke pred for each pair
    if oov set Ke to oov  # after test all in voc

    composition method
    add or pointwise
    '''

    n = len(listA)
    if method == 'add':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] + dic[phraseA[1].lower()] + dic[phraseA[2].lower()]
            newB = dic[phraseB[0].lower()] + dic[phraseB[1].lower()] + dic[phraseB[2].lower()]
            Ke_list.append(ke(newA, newB))

    if method == 'pointwise':
        Ke_list = []
        for i in range(n):
            phraseA = listA[i].split()
            phraseB = listB[i].split()
            newA = dic[phraseA[0].lower()] * dic[phraseA[1].lower()] * dic[phraseA[2].lower()]
            newB = dic[phraseB[0].lower()] * dic[phraseB[1].lower()] * dic[phraseB[2].lower()]
            Ke_list.append(ke(newA, newB))
    return Ke_list


# get sv dataset
dicA = []
dicB = []
val = []
with open("F:/MSc DS/Project/compositional datasets/KS2016-SVO.txt") as f:
    for line in f:
        linelist = line.split(",")
        dicA.append(linelist[0])
        dicB.append(linelist[1])
        val.append(linelist[2])


# convert the T to 1 F to 0 in val list
n = len(val)
for i in range(n):
    if val[i]=='T\n':
        val[i]=1
    else:
        val[i]=0
# dicA  list to store the first phrase
# dicB  list to store the second pharse
# val  true val


In [2]:
# test different source with different composition methods
dic1 = get_dict_from_source(source1)

In [7]:
for i in range(len(dicA)):
    for j in dicA[i].split():
        if j.lower() not in dic1.keys():
            print(i,j)

In [6]:
for i in range(len(dicB)):
    for j in dicB[i].split():
        if j.lower() not in dic1.keys():
            print(i,j)

In [5]:
dicA.pop(22)
dicB.pop(22)
val.pop(22)
dicA.pop(22)
dicB.pop(22)
val.pop(22)

0

In [8]:
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.46439823566477634
0.4864524259609326


In [9]:
dic1 = get_dict_from_source(source2)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.5082965763495064
0.5826507036336904


In [10]:
dic1 = get_dict_from_source(source3)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4675488342785129
0.5103969754253308


In [11]:
dic1 = get_dict_from_source(source4)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.5106070153329132
0.589582020583911


In [12]:
dic1 = get_dict_from_source(source5)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4849821466078555
0.38395295106070154


In [13]:
dic1 = get_dict_from_source(source6)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.47490023104389834
0.46334803612686415


In [14]:
dic1 = get_dict_from_source(source7)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.4782608695652173
0.5015752993068683


In [18]:
dic1 = get_dict_from_source(source8)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.47258979206049145
0.40747742070993487


In [16]:
dic1 = get_dict_from_source(source9)
# add method
pred = compute_Ke(dicA, dicB,dic1)
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

# pointwise
pred = compute_Ke(dicA, dicB,dic1,method='pointwise')
from sklearn.metrics import roc_auc_score
result = roc_auc_score(val, pred)
print(result)

0.5040957781978576
0.49233354337324087
